In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
X=np.load(r'C:\Users\varad\OneDrive\Desktop\datasets\AndreNg\unsupervised learning\collaborativefiltering\X.npy')

In [54]:
X[:,5],X.shape

(<tf.Tensor: shape=(4778,), dtype=float64, numpy=
 array([ 0.85633146, -0.06540668,  0.50724804, ..., -1.3004357 ,
         1.47717244,  0.62241401])>,
 TensorShape([4778, 100]))

In [3]:
Y=np.load(r'C:\Users\varad\OneDrive\Desktop\datasets\AndreNg\unsupervised learning\collaborativefiltering\Y.npy')

In [55]:
Y[:,5],Y.shape

(array([0., 0., 0., ..., 0., 0., 0.]), (4778, 443))

In [67]:


Ymean = tf.reduce_sum(Y, axis=0) / tf.reduce_sum(R, axis=0) 
Ymean = tf.where(tf.math.is_nan(Ymean), tf.zeros_like(Ymean), Ymean)  

Ynorm = (Y - Ymean) * R 


In [68]:
Ynorm.shape

TensorShape([4778, 443])

In [4]:
R=np.load(r'C:\Users\varad\OneDrive\Desktop\datasets\AndreNg\unsupervised learning\collaborativefiltering\R.npy')

In [51]:
R[:,5],R.shape

(array([0., 0., 0., ..., 0., 0., 0.]), (4778, 443))

In [5]:
b=np.load(r'C:\Users\varad\OneDrive\Desktop\datasets\AndreNg\unsupervised learning\collaborativefiltering\b.npy')

In [18]:
b[0][:5]

array([0.23609531, 0.35653394, 0.12819964, 0.3155024 , 0.23636407])

In [50]:
b.shape

TensorShape([1, 443])

In [6]:
W=np.load(r'C:\Users\varad\OneDrive\Desktop\datasets\AndreNg\unsupervised learning\collaborativefiltering\W.npy')

In [53]:
W.shape

TensorShape([443, 100])

In [21]:
num_features =10
num_movies =4778
num_users 443

In [22]:
def cofi_cost_func(X, W, b, Y, R, lambda_):
    nm, nu = Y.shape
    J = 0
    R1=0
    R2=0
    nf=X.shape[1]
    for i in range(nm):
        for j in range(nu):
            if R[i][j]==1:
                J+=0.5*((np.dot(W[j],X[i])+b[0][j])-Y[i][j])**2
                
            else:
                pass
    for i in range(nm):
        for k in range(nf):
            R1+=(X[i][k])**2
    for i in range(nu):
        for k in range(nf):
            R2+=(W[i][k])**2
    
    regularization= lambda_*0.5*(R1+R2)

    return J+regularization

In [23]:
num_users_r = 4
num_movies_r = 5 
num_features_r = 3

X_r = X[:num_movies_r, :num_features_r]
W_r = W[:num_users_r,  :num_features_r]
b_r = b[0, :num_users_r].reshape(1,-1)
Y_r = Y[:num_movies_r, :num_users_r]
R_r = R[:num_movies_r, :num_users_r]

# Evaluate cost function
J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Cost: {J:0.2f}")

Cost: 13.67


In [24]:
J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

Cost (with regularization): 28.09


# Vectorization

In [45]:
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    pred = pred = tf.matmul(X, tf.transpose(W)) + tf.reshape(b, (1, -1))

    j = (pred - Y) * R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [46]:
J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Cost: {J:0.2f}")

J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

Cost: 13.67
Cost (with regularization): 28.09


In [47]:
num_movies, num_users = Y.shape
num_features = 100

tf.random.set_seed(1234) 
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')


optimizer = keras.optimizers.Adam(learning_rate=1e-1)

In [48]:
print("X @ W.T:", tf.linalg.matmul(X, tf.transpose(W)).shape)
print("b:", b.shape)
print("Y:", Y.shape)


X @ W.T: (4778, 443)
b: (1, 443)
Y: (4778, 443)


In [69]:
iterations = 200
lambda_ = 1
for iter in range(iterations):
    with tf.GradientTape() as tape:

        cost_value = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)

    grads = tape.gradient( cost_value, [X,W,b] )

    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 2289476.3
Training loss at iteration 20: 133286.3
Training loss at iteration 40: 50533.8
Training loss at iteration 60: 24009.2
Training loss at iteration 80: 13394.1
Training loss at iteration 100: 8449.8
Training loss at iteration 120: 5885.6
Training loss at iteration 140: 4458.9
Training loss at iteration 160: 3624.9
Training loss at iteration 180: 3118.0


In [71]:
my_rated=[246, 366, 382, 622, 793, 929, 988, 1150, 2609, 2700, 2716, 2925, 2937]

In [73]:
movieList=np.load(r'C:\Users\varad\OneDrive\Desktop\datasets\AndreNg\unsupervised learning\collaborativefiltering\movieList.npy')

In [75]:
my_ratings=np.load(r'C:\Users\varad\OneDrive\Desktop\datasets\AndreNg\unsupervised learning\collaborativefiltering\my_ratings.npy')

In [76]:
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

#restoring mean
pm = p + Ymean

my_predictions = pm[:,0]

# sort predictions
ix = tf.argsort(my_predictions, direction='DESCENDING')

for i in range(17):
    j = ix[i]
    if j not in my_rated:
        print(f'Predicting rating {my_predictions[j]:0.2f} for movie {movieList[j]}')

print('\n\nOriginal vs Predicted ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print(f'Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {movieList[i]}')

Predicting rating 5.30 for movie Hot Fuzz (2007)
Predicting rating 5.29 for movie Juno (2007)
Predicting rating 5.27 for movie Blow (2001)
Predicting rating 5.23 for movie Interstellar (2014)
Predicting rating 5.15 for movie Departed, The (2006)
Predicting rating 5.14 for movie X2: X-Men United (2003)
Predicting rating 5.13 for movie Coco (2017)
Predicting rating 5.11 for movie Star Trek (2009)
Predicting rating 5.08 for movie Howl's Moving Castle (Hauru no ugoku shiro) (2004)
Predicting rating 5.05 for movie Zombieland (2009)
Predicting rating 5.04 for movie My Big Fat Greek Wedding (2002)
Predicting rating 5.03 for movie Harry Potter and the Goblet of Fire (2005)
Predicting rating 5.03 for movie Superbad (2007)
Predicting rating 5.01 for movie Pursuit of Happyness, The (2006)
Predicting rating 5.01 for movie Prestige, The (2006)


Original vs Predicted ratings:

Original 5.0, Predicted 4.92 for Shrek (2001)
Original 5.0, Predicted 4.69 for Harry Potter and the Sorcerer's Stone (a.k.a